# 페이스북 크롤링

In [1]:
# 페이지 스크롤 횟수 설정
num_scrolldown = 3

In [2]:
from bs4 import BeautifulSoup as bs
import time
import random
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
import pandas as pd

In [4]:
# # 로그인 페이지 접속(2단계 인증으로 로그인 불가능)

# # 로그인 정보
# username = 'global.gyeonggido@gmail.com'
# password = 's80083044!'

# element_id = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[1]/label/input')
# element_id.send_keys(username)
# element_password = driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[2]/label/input')
# element_password.send_keys(password)
# time.sleep(1.5)

# ##로그인버튼 클릭
# driver.find_element_by_xpath('//*[@id="login_form"]/div[2]/div[3]/div/div/div[1]/div/span/span').click()
# time.sleep(3)

In [5]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(3)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)
driver.implicitly_wait(5)

In [6]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.implicitly_wait(5)

In [7]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print(soup.title.text)

# 팔로워 수 수집
# num_follower = soup.find('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m')

# 글 날짜 수집
info_dates_span = soup.find_all('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw')

# 글 내용 크롤링
content_pages = soup.find_all('div', class_='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')

# 좋아요, 댓글, 공유 수 정보 나온 페이지 수집
info_pages = soup.find_all('div', 'bp9cbjyn m9osqain j83agx80 jq4qci2q bkfpd7mw a3bd9o3v kvgmc6g5 wkznzc2l oygrvhab dhix69tm jktsbyx5 rz4wbd8a osnr6wyh a8nywdso s1tcr66n')

print("날짜 수집 개수 : ", len(info_dates_span))
print("글 내용 수집 개수 : ", len(content_pages))
print("각종 정보 수집 개수 : ", len(info_pages))

# chromedriver 종료
driver.quit()

Gyeonggi Korea | Facebook
날짜 수집 개수 :  14
글 내용 수집 개수 :  13
각종 정보 수집 개수 :  13


## 글 날짜, 글 내용, 좋아요 수, 댓글 수, 공유 수 리스트화

### 날짜 수집

In [8]:
# 날짜 수집
num_date_list = []
for date in info_dates_span:
    if date.span != None:
        temp = date.span.text
        num_date_list.append(temp)

print('날짜 리스트 원소수 : ',len(num_date_list))

날짜 리스트 원소수 :  13


In [9]:
# 수집한 날짜 리스트 확인
len(num_date_list), num_date_list

(13,
 ['1시간',
  '어제 오후 2:00',
  '3월 15일 오후 4:14',
  '3월 12일 오후 2:00',
  '3월 11일 오후 1:19',
  '3월 10일 오후 2:52',
  '3월 8일 오후 6:05',
  '3월 4일 오후 6:21',
  '3월 4일 오전 10:58',
  '3월 2일 오후 5:04',
  '2월 28일 오후 5:49',
  '2월 25일 오전 10:19',
  '2월 24일 오후 3:00'])

In [10]:
# 날짜 형식 변환해서 리스트로 저장
mode_num_date_list = []
for date in num_date_list:
    if '시간' in date:
        temp = datetime.today().strftime("%m-%d")
        mode_num_date_list.append(temp)
    elif '어제' in date:
        temp = datetime.today().strftime("%m-%d")
        temp = temp.split('-')
        month = temp[0]
        day = int( temp[1] ) - 1
        temp_date = f'{month:0>2}-{day:0>2}'
        mode_num_date_list.append(temp_date)
    else:
        temp = date.split('일')[0]
        temp = temp.replace(' ', '')
        temp_li = temp.split('월')
        month = temp_li[0]
        day = temp_li[1]
        temp_date = f'{month:0>2}-{day:0>2}'
        mode_num_date_list.append(temp_date)

len(mode_num_date_list), mode_num_date_list

(13,
 ['03-17',
  '03-16',
  '03-15',
  '03-12',
  '03-11',
  '03-10',
  '03-08',
  '03-04',
  '03-04',
  '03-02',
  '02-28',
  '02-25',
  '02-24'])

### 글 내용, 좋아요 수, 댓글 수, 공유 수 수집

In [11]:
# 글 내용 수집
contents_list = []
for content in content_pages:
    temp = content.text
    contents_list.append(temp)

# 좋아요 수 수집
num_like_list = []
for like in info_pages:
    temp = like.find('span', 'pcp91wgn')
    temp = int(temp.text)
    num_like_list.append(temp)

# 댓글 수, 공유 수 수집
num_comment_list = []
num_share_list = []
for comment in info_pages:
    temp = comment.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
    if len(temp) == 1:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
                num_share_list.append(0)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
                num_comment_list.append(0)
    elif len(temp) == 2:
        for one in temp:
            if '댓글' in one.text:
                comment_one = int(one.text[3:-1])
                num_comment_list.append(comment_one)
            elif '공유' in one.text:
                comment_share = int(one.text[3:-1])
                num_share_list.append(comment_share)
    else:
        num_share_list.append(0)
        num_comment_list.append(0)

# 게시글 링크 수집
link_list = []
raw_link = soup.find_all('span', class_='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn mdeji52x e9vueds3 j5wam9gi b1v8xokw m9osqain hzawbc8m')
for link in raw_link:
    temp_link = link.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('?')[0]
    link_list.append(temp_link)


print('글 내용 리스트 원소수 : ',len(contents_list))
print('좋아요 수 리스트 원소수 : ',len(num_like_list))
print('공유 수 리스트 원소수 : ',len(num_share_list))
print('댓글 수 리스트 원소수 : ',len(num_comment_list))
print('링크 리스트 원소수 : ',len(link_list))

글 내용 리스트 원소수 :  13
좋아요 수 리스트 원소수 :  13
공유 수 리스트 원소수 :  13
댓글 수 리스트 원소수 :  13
링크 리스트 원소수 :  13


In [12]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':mode_num_date_list, '글_내용':contents_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,글_내용,좋아요_수,공유수,댓글수,링크
0,03-17,Successful Overseas Expansion by Local Startup...,6,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,03-16,[2022 Gyeonggi Foreign Social Media Reporters ...,29,3,1,https://www.facebook.com/GyeonggiKorea.en/post...
2,03-15,Gyeonggi to produce and distribute tourism bro...,40,1,2,https://www.facebook.com/GyeonggiKorea.en/post...
3,03-12,Gyeonggi Province’s Suwon City creates “Foreig...,22,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,03-11,Catch this week's #GTV English #News for the l...,10,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,03-10,Gyeonggi Province's Hanam City home to new LG-...,22,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,03-08,Dongducheon Rock Festival selected as an outst...,8,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
7,03-04,Gyeonggi Province’s Pocheon City provides fore...,22,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,03-04,Catch this week's #GTV English #News for the l...,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,03-02,Gyeonggi Province promotes local tourism with ...,22,2,0,https://www.facebook.com/GyeonggiKorea.en/post...


In [13]:
# 인덱스 내림차순 정렬
final_data = data.sort_index(ascending=False)
final_data

,날짜,글_내용,좋아요_수,공유수,댓글수,링크
12,02-24,Hwaseong Fortress of Suwon City,50,10,0,https://www.facebook.com/GyeonggiKorea.en/post...
11,02-25,Catch this week's #GTV English #News for the l...,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
10,02-28,Gyeonggi Province is taking the initiative in ...,53,11,0,https://www.facebook.com/GyeonggiKorea.en/post...
9,03-02,Gyeonggi Province promotes local tourism with ...,22,2,0,https://www.facebook.com/GyeonggiKorea.en/post...
8,03-04,Catch this week's #GTV English #News for the l...,11,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
7,03-04,Gyeonggi Province’s Pocheon City provides fore...,22,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
6,03-08,Dongducheon Rock Festival selected as an outst...,8,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
5,03-10,Gyeonggi Province's Hanam City home to new LG-...,22,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,03-11,Catch this week's #GTV English #News for the l...,10,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
3,03-12,Gyeonggi Province’s Suwon City creates “Foreig...,22,0,0,https://www.facebook.com/GyeonggiKorea.en/post...


In [14]:
# 수집한 데이터 파일료 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = './crawling_data/facebook/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

facebook_20220317-163412.xlsx 파일 생성 완료


# 코드 마지막